In [62]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Loading in a Saved Model

In [63]:
# Load model
loaded_SaveModel_format = tf.keras.models.load_model("best_model_SavedModel_format")
loaded_SaveModel_format.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                20        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 31 (124.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
X = tf.range(-100, 100, 4)

y = X + 10

X, y

(<tf.Tensor: shape=(50,), dtype=int32, numpy=
 array([-100,  -96,  -92,  -88,  -84,  -80,  -76,  -72,  -68,  -64,  -60,
         -56,  -52,  -48,  -44,  -40,  -36,  -32,  -28,  -24,  -20,  -16,
         -12,   -8,   -4,    0,    4,    8,   12,   16,   20,   24,   28,
          32,   36,   40,   44,   48,   52,   56,   60,   64,   68,   72,
          76,   80,   84,   88,   92,   96])>,
 <tf.Tensor: shape=(50,), dtype=int32, numpy=
 array([-90, -86, -82, -78, -74, -70, -66, -62, -58, -54, -50, -46, -42,
        -38, -34, -30, -26, -22, -18, -14, -10,  -6,  -2,   2,   6,  10,
         14,  18,  22,  26,  30,  34,  38,  42,  46,  50,  54,  58,  62,
         66,  70,  74,  78,  82,  86,  90,  94,  98, 102, 106])>)

In [65]:
X_train = X[:40]
y_train = y[:40]

X_test = X[40:]
y_test = y[40:]

X_train, y_train, X_test, y_test

(<tf.Tensor: shape=(40,), dtype=int32, numpy=
 array([-100,  -96,  -92,  -88,  -84,  -80,  -76,  -72,  -68,  -64,  -60,
         -56,  -52,  -48,  -44,  -40,  -36,  -32,  -28,  -24,  -20,  -16,
         -12,   -8,   -4,    0,    4,    8,   12,   16,   20,   24,   28,
          32,   36,   40,   44,   48,   52,   56])>,
 <tf.Tensor: shape=(40,), dtype=int32, numpy=
 array([-90, -86, -82, -78, -74, -70, -66, -62, -58, -54, -50, -46, -42,
        -38, -34, -30, -26, -22, -18, -14, -10,  -6,  -2,   2,   6,  10,
         14,  18,  22,  26,  30,  34,  38,  42,  46,  50,  54,  58,  62,
         66])>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([60, 64, 68, 72, 76, 80, 84, 88, 92, 96])>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 70,  74,  78,  82,  86,  90,  94,  98, 102, 106])>)

#### Compare model 2 predictions with SaveModel format model predictions

In [66]:
# Let's build a model

tf.random.set_seed(42)

# 1. Create a Model
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
], name="TensorFlow_39")

# 2. Compile a model
model_2.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.SGD(),
                metrics=['mae'])

# 3. Fit the model
model_2.fit(tf.expand_dims(X_train, axis= -1), y_train, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 4ms/step - loss: 50.0365 - mae: 50.0365
Epoch 2/100
2/2 [==============================] - 0s 1ms/step - loss: 32.1000 - mae: 32.1000
Epoch 3/100
2/2 [==============================] - 0s 2ms/step - loss: 36.8310 - mae: 36.8310
Epoch 4/100
2/2 [==============================] - 0s 2ms/step - loss: 28.6624 - mae: 28.6624
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 15.5530 - mae: 15.5530
Epoch 6/100
2/2 [==============================] - 0s 1ms/step - loss: 12.3418 - mae: 12.3418
Epoch 7/100
2/2 [==============================] - 0s 2ms/step - loss: 11.6308 - mae: 11.6308
Epoch 8/100
2/2 [==============================] - 0s 2ms/step - loss: 12.0521 - mae: 12.0521
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 37.6881 - mae: 37.6881
Epoch 10/100
2/2 [==============================] - 0s 1ms/step - loss: 25.3271 - mae: 25.3271
Epoch 11/100
2/2 [==============================] - 0s 2ms/

In [67]:
# Compare predictions
model_2_preds = model_2.predict(X_test)
loaded_SavedModel_format_preds = loaded_SaveModel_format.predict(X_test)
model_2_preds == loaded_SavedModel_format_preds

1/1 [==============================] - 0s 27ms/step


array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])

In [68]:
# Create Evaluation function 
def mae(y_true, y_pred):
    return tf.metrics.mean_absolute_error(y_true, tf.squeeze(y_pred))

def mse(y_true, y_pred):
    return tf.metrics.mean_squared_error(y_true, tf.squeeze(y_pred))

In [69]:
mae(y_true=y_test, y_pred=model_2_preds) == mae(y_true=y_test, y_pred=loaded_SavedModel_format_preds)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [70]:
model_2_preds

array([[ 78.275185],
       [ 83.37551 ],
       [ 88.47583 ],
       [ 93.576164],
       [ 98.67648 ],
       [103.77682 ],
       [108.87713 ],
       [113.97746 ],
       [119.07779 ],
       [124.178116]], dtype=float32)

In [71]:
loaded_SavedModel_format_preds

array([[ 69.60265 ],
       [ 74.13006 ],
       [ 78.657455],
       [ 83.18487 ],
       [ 87.712265],
       [ 92.23967 ],
       [ 96.76707 ],
       [101.29447 ],
       [105.82187 ],
       [110.34928 ]], dtype=float32)

In [72]:
model_2_preds.squeeze()

array([ 78.275185,  83.37551 ,  88.47583 ,  93.576164,  98.67648 ,
       103.77682 , 108.87713 , 113.97746 , 119.07779 , 124.178116],
      dtype=float32)

In [73]:
loaded_SavedModel_format_preds.squeeze()

array([ 69.60265 ,  74.13006 ,  78.657455,  83.18487 ,  87.712265,
        92.23967 ,  96.76707 , 101.29447 , 105.82187 , 110.34928 ],
      dtype=float32)

In [74]:
model_2_preds.squeeze() == loaded_SavedModel_format_preds.squeeze()

array([False, False, False, False, False, False, False, False, False,
       False])